In [ ]:
import pandas as pd
import openai
from keybert import KeyBERT
import langdetect
import time
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import os 
import glob

from transformers import pipeline

pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("display.max_colwidth", 200)

# model = KeyBERT('distilbert-base-nli-mean-tokens')

In [ ]:
with open("apikey_moritz.txt", "r") as f:
    api_key = f.read()

print(api_key)
openai.api_key = api_key

## Text: Finetuned GPT3 Model


In [ ]:
fine_tune_list = openai.FineTune.list()
fine_tuned_model = fine_tune_list['data'][0].fine_tuned_model
print(fine_tuned_model)

In [ ]:
new_prompt = "A Google Maps review about a steak house. ###"
answer = openai.Completion.create(
  model=fine_tuned_model,
  prompt=new_prompt,
  max_tokens=200, # Change amount of tokens for longer completion
  temperature=1,
  stop = "END"
)
print(answer['choices'][0]['text'])

In [ ]:
def gpt3_finetuned(row):
    catgeoryName = row['categoryName'].lower()
    prompt = "A Google Maps review about a " + catgeoryName + ". ###"
    print(prompt)
    answer = openai.Completion.create(
      model=fine_tuned_model,
      prompt=prompt,
      max_tokens=200, # Change amount of tokens for longer completion
      temperature=1, 
      stop = "END"
    )
    return answer['choices'][0]['text']

# def gpt3_5_turbo(row):
#     prompt = "Write a Google Maps review about a " + row['categoryName'] + ". In Style of a Google user that writes a review about a place."
#     role = "In stlye of a google user that writes a review about a place."
#     prompt = "Write a Google Maps review about a " + row['categoryName']+"."
#     print(prompt)
#     return chatGPT(api_key, role).question(prompt)

#### Outdated: Nur für Test:  gpt3_5_turbo vs gpt3_finetuned 

In [ ]:
# ## df_test.to_csv('test_df_showcase20gpt.csv')
# # print("Done with gpt3_finetuned_v1 text generation.")

# df_test = pd.read_csv('test_df_showcase.csv')
# df_test['gpt3_finetuned_v1'] = df_test.apply(gpt3_finetuned, axis=1)
# df_test.to_csv('test_df_showcase.csv')
# print("Done with gpt3.5_turbo text generation.")

# print(df_test)
# print(df_test.shape)

In [ ]:
# df_test = pd.read_csv('test_df_showcase.csv')
# df_test['gpt3.5_turbo'] = df_test['gpt3.5_turbo'].astype(str)

# for index, row in df_test.iterrows():
#     print(row['title'])
#     print(row['categoryName'])
#     print("ORG:" + row['original_text'])
#     print("GPT:" + row['gpt3_finetuned_v1'])
#     print("GPT3.5:" + row['gpt3.5_turbo'])
#     print("__________________________________________________________________________________________")

### Jetzt tatsächliche Daten augmentiert / generiert. Dazu einen neuen Dataframe erstellen...

Später wird der dann bemerged.

In [ ]:
# df_true = pd.read_csv('base_keywords_sentiment_reduced.csv')
# df_finetune = pd.read_csv('dataset_ft_v3_en_mulitmod_pc.csv.csv')

# # print(df_true['categoryName'].value_counts())
# # print(df_finetune['categoryName'].value_counts())
# print(f'True:' , df_true.shape)

# df_fake = pd.DataFrame(columns=df_true.columns)
# df_fake['categoryName'] = df_true['categoryName']
# df_fake['label'] = 'FAKE'

# # just for reference. Can be delted later on. 
# df_fake['reviewId'] = df_true['reviewId']
# df_fake['title'] = df_true['title']
# df_fake['text'] = df_true['text']
# df_fake['placeId'] = df_true['placeId']

# df_fake.to_csv('fake_base.csv', index=False)

#### Ab hier neue finetunes ausprobieren und unter neuer Version speichern. 
 Wichtig! Spaltennamen ändern von gpt3_v3

In [ ]:
df_real = pd.read_csv('fake_base_gpt3_v2331.csv')
df_real["prompt_v4"] = "A Google Maps review about a " + df_real["categoryName"] + ". ###"

for index, row in df_real.iterrows():
    df_real.at[index, 'gpt3_v4'] = gpt3_finetuned(row)
    print(df_real.at[index, 'gpt3_v4'])
    print(index)
    if index % 50 == 0:
        df_real.to_csv('fake_base_gpt3_v23314.csv', index=False)
        print("Wir warten ein paar Sekunden und hoffen, dass der API key deshalb nicht geblockt wird...")
        time.sleep(25)
        print("Weiter geht's...")

print("Done with gpt3 text generation.")
print(df_real.shape)
print(df_real.columns)

df_real.to_csv("fake_base_gpt3_v23314.csv", index=False)

In [ ]:
# load df_fake and save in xlsx

df_real = pd.read_csv('fake_base_gpt3_v23314.csv')
print(df_real.shape)


# Image Generierung


Vorbereitung. Eine Sentiment-Analyse auf dem generierten Text durchführen. 

In [ ]:
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

In [ ]:
# do a sentiment analysis on the generated text gpt3_v2, gpt3_v3 gpt3_v3.1

df_real = pd.read_csv('fake_base_gpt3_v23314.csv')
df_real['sent_v2'] = np.nan
df_real['sent_v3']= np.nan
df_real['sent_v3.1'] = np.nan
df_real['sent_v4'] = np.nan

df_real['sent_v2'] = df_real['gpt3_v2'].astype(str)
df_real['sent_v3'] = df_real['gpt3_v3'].astype(str)
df_real['sent_v3.1'] = df_real['gpt3_v3.1'].astype(str)
df_real['sent_v4'] = df_real['gpt3_v4'].astype(str)

for index, row in df_real.iterrows():
    print("Processing row: ", index)
    try:
        df_real.at[index, 'sent_v2'] = sentiment_task(row['gpt3_v2'])[0]['label']
        df_real.at[index, 'sent_v3'] = sentiment_task(row['gpt3_v3'])[0]['label']
        df_real.at[index, 'sent_v3.1'] = sentiment_task(row['gpt3_v3.1'])[0]['label']
        df_real.at[index, 'sent_v4'] = sentiment_task(row['gpt3_v4'])[0]['label']
    except RuntimeError:
        print("Skipped due to messy data. Row: ", index)
        pass

df_real.to_csv('fake_base_gpt3_v23314_sent.csv', index=False)
df_real.to_excel('fake_base_gpt3_v23314_sent.xlsx', index=False)


## Image: Dall-E-2


In [ ]:
with open("apikey_papa.txt", "r") as f:
    openai.api_key = f.read()

In [ ]:
df_real = pd.read_csv('fake_base_gpt3_v23314_sent.csv')
print(df_real.shape)
print(df_real.columns)
print(df_real['categoryName'])

In [ ]:
def generate_dalle_prompt(row):
    genre = row["genre"].lower()
    categoryName = row["categoryName"].lower()
    sentiment = row["sent_v4"].lower()
    locationString = f"a {categoryName}"
    tone = ""
    if sentiment == "positive":
        sentiment = "great"
        inspire_sentence = "This photo is sure to delight and inspire anyone who sees it"
        if genre == "restaurant":
            sentiment = "delicious"
            tone = "delicious, well-decorated"
        elif genre == "hotel":
            sentiment = "great"
            tone = "clean"
        elif genre == "activity":
            inspire_sentence = "This photo is sure to inspire anyone who sees it to do the same activity."
            sentiment = "fun"
            tone = "fun, happy athmosphere"
    elif sentiment == "negative":
        inspire_sentence = "This photo is likely to disappoint and demotivate anyone who sees it"
        sentiment = "dissappointing"
        if genre == "restaurant":
            tone = "bad service, dirty, bad decoration"
        elif genre == "hotel":
            tone = "dirty, uncomfortable"
        elif genre == "activity":
            tone = "boring, lame"
    else:
        inspire_sentence = ""
        sentiment = "average"

    if genre == "restaurant":
       # PROMPT = f"A makro of {sentiment} food at a {categoryName}, {tone}, {inspire_sentence}, Sigma 105mm F2.8, close-up, hyper detailed, Super-Resolution, UHD, DTM, HDR, 8K --ar 4:3 --q 2, on Tripadvisor.com in 2022"
        PROMPT =  f"A photo of {sentiment} food at a {categoryName}. {inspire_sentence}. The shot is taken with a Canon EOS 5D Mark IV camera and a 40mm f/1.8 lens, with settings of f/5.6, ISO 100, and a shutter speed of 1/200 sec. Close-up, Super-Resolution, UHD, DTM, HDR, 8K --ar 4:3 --q 2,  on Tripadvisor.com 2022"
    elif genre == "hotel":
        PROMPT = f"A photo of {sentiment} stay at a {categoryName}, {inspire_sentence}. Sigma lens, wide shot, full shot. hyper detailed, Super-Resolution, UHD, DTM, HDR, 8K --ar 4:3 --q 2, on Bookings.com in 2022"
    elif genre == "activity":
        PROMPT = f"A photo of a {categoryName} taken by a photograph visitor. The shot is taken with a Canon EOS 5D Mark IV camera and a 40mm f/1.8 lens, hyper detailed, Super-Resolution, UHD, DTM, HDR, 8K --ar 4:3 --q 2, on Tripadvisor.com in 2022"

    print(PROMPT)
    return PROMPT

prompt = generate_dalle_prompt(df_real.iloc[599])
response = openai.Image.create(prompt=prompt, n=1, size="256x256")
print(response["data"][0]["url"])


In [ ]:
df_real = pd.read_csv('fake_base_gpt3_v23314_sent_dalle.csv')
print(df_real.shape)

In [ ]:
df_real = pd.read_csv('fake_base_gpt3_v23314_sent_dalle.csv')

df_real['fake_reviewImageUrls/0'] = ""
print(df_real.shape)

for index, row in df_real.iterrows():
    if  index < 449:
        print("Schon berechnet (auf anderem Account): ", index)
        continue
    print("Generating Promt for row: ", index)
    prompt = generate_dalle_prompt(row)
    df_real.at[index, 'dalle_prompt'] = prompt
    print("Generated Image for index: ", index)
    response = openai.Image.create(prompt=prompt, n=1, size="256x256")
    df_real.at[index, 'fake_reviewImageUrls/0'] = response["data"][0]["url"]
    response_url = response["data"][0]["url"]
    print(response["data"][0]["url"])
    df_real.to_csv('fake_base_gpt3_v23314_sent_dalle.csv', index=False)
    print("Saved Image for index: ", index)
    print(f"_______________________NOW START DOWNLOADING FOR INDEX {index} ____________________________________________________________________")
    reviewId = row['reviewId']
    reviewIdF = row['reviewIdF']
    print(f"Downloading Image for index {index}, reviewId {reviewId}, reviewIdF {reviewIdF}...")
    print(row['fake_reviewImageUrls/0'])
    response = requests.get(response_url)
    img = Image.open(BytesIO(response.content))
    print(img.mode)
    print(img.size)
    img.save(f'fake_image_url_0/{index}_{reviewId}_fake_reviewImageUrls_0.png')
    print("Saved Image for index: ", index)
    print("_______________________________DONE___________________________________________________________")

print("Fertig mit Erzeugung von Bildern mit API über Dalle 2.")

Eveentuell die erzeugten Bilder umbennen, falls System was aus Filenamen lernt bzw es die Filenamen nicht mehr passen wegen dem zusammengeführten Index später

...rename a file from folder fake_image_url_0


In [ ]:
# for filename in glob.glob('fake_image_url_0/*.png'):
#     print(filename)
#     os.rename(filename, filename[4:])

In [ ]:
pd.set_option('display.max_colwidth', None)
df_real = pd.read_csv('fake_base_gpt3_v23314_sent_dalle.csv')
df_real['githubUrls/0'] = ""
print(df_real.shape)

for index, row in df_real.iterrows():
    print("Generating Github Url for row: ", index)
    github_path = f"https://raw.githubusercontent.com/MichaelSeitz98/seminararbeit_review_detection/main/02_Images/fake_image_url_0/{index}_{row['reviewId']}_fake_reviewImageUrls_0.png"
    df_real.at[index, 'githubUrls/0'] = github_path
    print(f"Generated {github_path} Github Url for index: ", index)

print("Fertig mit Erzeugung von Github Urls.")
print(df_real["githubUrls/0"])

df_real.to_csv('fake_base_gpt3_v23314_sent_dalle.csv', index=False)

In [ ]:
df_real = pd.read_csv('fake_base_gpt3_v23314_sent_dalle.csv')
print(df_real.columns)
df_real.to_excel('fake_base_gpt3_v23314_sent_dalle.xlsx', index=False)

# Tabulare Daten 

Fake publishedAtDate & publishedAt Column im Fake dataset and store it in column published_at_date_column

In [ ]:
from faker import Faker
import pytz
from datetime import datetime, timedelta

fake = Faker('en_GB')

print(df_real['publishedAtDate'].min())
print(df_real['publishedAtDate'].max())
end_date = datetime(2023, 3, 30)
start_date = end_date - timedelta(days=365*7)

fake_time = fake.date_time_between(start_date=start_date, end_date=end_date, tzinfo=None)
fake_time_str = fake_time.strftime('%Y-%m-%dT%H:%M:%S')
print(fake_time_str)


Fake the name column with faker and store it in column name of df_fake

likesCount, reviewerNumberOfReviews, isLocalGuide mit TabGAN oder CTGAN erzeugen